In [ ]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import openai
import os
import numpy as np

# OpenAI API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]  # 환경 변수에서 불러오는 것이 더 안전함

# 문장을 임베딩으로 변환하는 함수
def sentence_to_embedding(sentence):
    response = openai.embeddings.create(
        input=sentence,
        model="text-embedding-ada-002"  # OpenAI embedding 모델
    )
    return np.asarray(response.data[0].embedding, dtype=np.float32)  # ndarray Type 벡터 반환

In [75]:
# 비교할 문장과 연산자(키워드) 리스트
texts = [
    "나는 자동차를 좋아합니다.",
    "나는 딸기를 싫어합니다.",
    "나는 자동차를 싫어합니다.",
    "나는 딸기를 좋아합니다.",
    "한국에는 산이 많습니다."
]

operators = [
    "자동차",
    "딸기",
    "무엇인가를 좋아하다",
    "무엇인가를 싫어하다",
]

t_text = [{"text": s, "embedding":sentence_to_embedding(s)} for s in texts]
o_embs = {s:sentence_to_embedding(s) for s in operators}

In [ ]:


# 원래 문장
sentence = t_text[0]  # "나는 자동차를 좋아합니다."
print(f"원래 문장 : {sentence['text']}")

weight1 = 1  # 목적어 가중치 설정
weight2 = 1  # 동사 가중치 설정
embedding = sentence["embedding"]\
     + (o_embs["딸기"] - o_embs["자동차"]) * weight1 + \
     + (o_embs["무엇인가를 싫어하다"] - o_embs["무엇인가를 좋아하다"]) * weight2

embedding = embedding / np.linalg.norm(embedding, ord=2)  # 정규화

resutls = []
for i, s in enumerate(t_text):
    resutls.append({"id":i, "text":s['text'], "similarity":cosine_similarity([embedding], [s['embedding']])[0][0]})

resutls = sorted(resutls, key=lambda x: x["similarity"], reverse=True)

print("가장 유사한 문장 순위:")
for r in resutls:
    print(f"{r['id']}. {r['text']} (유사도: {r['similarity']:.4f})")

원래 문장 : 나는 자동차를 좋아합니다.
가장 유사한 문장 순위:
1. 나는 딸기를 싫어합니다. (유사도: 0.8949)
3. 나는 딸기를 좋아합니다. (유사도: 0.8526)
2. 나는 자동차를 싫어합니다. (유사도: 0.8154)
0. 나는 자동차를 좋아합니다. (유사도: 0.7736)
4. 한국에는 산이 많습니다. (유사도: 0.7433)
